# This notebook saves cafe v3 control daily climatologies in a single dataset

#### IMPORTANT: If adapting this code to save other climatologies, be careful to only include full years, as pyLatte will compute monthly climatologies from the saved daily climatologies

In [1]:
import pandas as pd
import xarray as xr
import numpy as np
from ipywidgets import FloatProgress
from dateutil.relativedelta import relativedelta

#### Only use last 100 years

In [60]:
base = '/g/data1/v14/coupled_model/v3/OUTPUT/'
years = range(400,500)

paths = []
for year in years:
    path = base + 'atmos_daily*' + str(year) + '*.nc'
    paths.append(path)
#da = xr.open_dataset(path, autoclose=True)['sst']

# da = xr.open_dataset(path, autoclose=True, decode_times=False)['sst']
# time_units = 'days since ' + str(year_shift) + '-01-01'
# decoded_time = xr.coding.times.decode_cf_datetime(da.time,time_units)
# shifted_time = np.array([np.datetime64(time - relativedelta(years=year_shift)).astype('datetime64[ns]') 
#                          for time in decoded_time])
# da.coords['time'] = ('time', utils.trunc_time(shifted_time,'M'), 
#                      {'long_name' : 'time', 'decoded_using' : time_units })

#time_use = da['time'][-14:-2].values

In [61]:
da = da.where(da>-800)

In [62]:
da_clim = da.groupby('time.month').mean(dim='time',keep_attrs=True).compute()
da_clim['month'] = time_use
da_clim = da_clim.rename({'month' : 'time'})
da_clim.time.attrs['long_name'] = 'time'
da_clim.time.attrs['cartesian_axis'] = 'T'
da_clim.time.attrs['calendar_type'] = 'JULIAN'
da_clim.time.attrs['bounds'] = 'time_bounds'

# Save and delete -----
savename = 'HadISST.1870011612_2018021612.clim.nc'

with utils.timer():
    da_clim.to_netcdf(path='/OSM/CBR/OA_DCFP/data/intermediate_products/pylatte_climatologies/' + savename,
                      mode = 'w',
                      encoding = {'time':{'dtype':'float','calendar':'JULIAN',
                                          'units':'days since 0001-01-01 00:00:00'}}) 

   Elapsed: 0.21576285362243652 sec
